<a href="https://colab.research.google.com/github/Indranil-R/Silver-Badge-Assignments/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Implement a question answering system with RAG, word embedding, vector database, langchain, llm and any other tools

In [4]:
!pip install -qU -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00


In [5]:
from os import getenv
from langchain_google_genai import GoogleGenerativeAI
from google import genai

In [9]:
from google.colab import userdata

In [10]:
llm = GoogleGenerativeAI(model="models/gemini-2.0-flash",google_api_key=userdata.get('GOOGLE_API_KEY'))
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

## Pros and Cons of Python as a Programming Language

Python is a versatile and popular programming language known for its readability and extensive libraries. However, like any language, it has its strengths and weaknesses. Here's a breakdown of the pros and cons:

**Pros:**

*   **Readability and Simplicity:**
    *   **Pro:** Python's syntax is designed to be clear and concise, making it easier to read and understand, even for beginners. It emphasizes code readability through indentation, reducing the need for excessive punctuation and brackets.
    *   **Benefit:** Faster development time, easier collaboration, and reduced maintenance costs.

*   **Large and Active Community:**
    *   **Pro:** Python boasts a massive and supportive community of developers. This translates to readily available resources, tutorials, forums, and libraries.
    *   **Benefit:** Easier to find solutions to problems, learn new skills, and get help when needed.

*   **Extensive Libraries and Frameworks:*

In [11]:
from langchain_core.prompts import PromptTemplate

In [23]:
template = """Question: {question}

Answer: What is the answer?."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "How much is 2+2?"

# print(chain.invoke({"question": question}))

input_variables=['question'] input_types={} partial_variables={} template='Question: {question}\n\nAnswer: What is the answer?.'


In [15]:
print(chain.invoke({"question": "now tell me 2+2"}))

The answer to 2 + 2 is 4.



In [21]:
import getpass
import os

from langchain.chat_models import init_chat_model

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

model.invoke("Please do not tel me about yourself!")

AIMessage(content="Understood. I won't tell you anything about myself. How can I help you with something else?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ae263354-25b9-4bbd-9200-879c504dca79-0', usage_metadata={'input_tokens': 8, 'output_tokens': 22, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}})

## Creating Embedding from Google API and storing into vector DB and then using it to retreive data

In [28]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

### Loading the data from Webpage

In [29]:
# I am fetching the data from SAP page due to API limitations, keeping the data small
loader = WebBaseLoader(
    web_paths=("https://www.sap.com/india/about/company/faq.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Grid__col-12--cMZPy Grid__col-sm-12--JrL2M Grid__col-md-12--tG5nF Grid__col-lg-12--wWsJK Grid__col-xl-12--v6LlR ")
        )
    ),
)
docs = loader.load()

In [30]:
print(docs)

[Document(metadata={'source': 'https://www.sap.com/india/about/company/faq.html'}, page_content='When was SAP founded?In 1972, five former IBM employees founded the company they call SAP Systemanalyse und Programmentwicklung ("System Analysis and Program Development").Taking the initial form of a private partnership under the German Civil Code, the company establishes its headquarters in Weinheim, Germany, and opens an office in nearby Mannheim. However, SAP\'s five founders spend most of their time in the data centres of their first customers, which include the German branch of Imperial Chemical Industries in Östringen.\nFurther information can be found on the\xa0SAP History page.What does the acronym "SAP" stand for?"SAP" stands for Systems, Applications, and Products in Data Processing.\nWhat is SAP’s vision and mission?We bring out the best in every business.\nThat\'s SAP\'s vision.\xa0\n\xa0\nTo turn this vision into reality for our customers, we have set ourselves the following t

In [31]:
!pip install -q langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:0

In [50]:
# from langchain.vectorstores.chroma import Chroma

### Creating the embedding

In [41]:
# Google EMbedding model is
# models/text-embedding-004

from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))

result = client.models.embed_content(
        model="text-embedding-004",
        contents="docs")


[ContentEmbedding(values=[0.056352835, -0.012802465, -0.04333053, -0.0062606214, 0.030048978, 0.021572424, 0.013492307, 0.03991041, 0.04568274, 0.032238908, -0.007996371, 0.060364235, 0.05384174, -0.008415636, -0.006721377, -0.03705883, 0.0008522501, 0.0089468, -0.058682784, -0.0035403266, 0.0024345098, -0.017475177, -0.059881948, -0.04279486, 0.037763897, -0.015532064, 0.06727401, 0.008493094, 0.0049022767, 0.0069087185, 0.0018617157, -0.0027591325, 0.021981413, -0.052339416, 0.026983641, -0.014949808, -0.03790051, 0.051977694, 0.048914775, -0.057983853, -0.0061779814, 0.035320774, -0.010377544, 0.009324791, 0.00076812477, 0.06585163, 0.0037922708, 0.025374575, -0.009712085, -0.011497512, 0.031240001, -2.4608615e-05, -0.024233906, -0.011671555, 0.03104733, -0.033371434, -0.053021085, -0.045064755, 0.06373168, 0.005851209, -0.016003685, 0.009296971, -0.010886275, 0.03319735, 0.01997865, 0.010300049, 0.021135475, -0.09178153, -0.041133057, 0.014314031, 0.013058472, 0.0032495563, -0.0080

### Adding the embedding to Chroma VectorDB

In [49]:

# check python ver
import sys
print(sys.version)


3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]


In [54]:
!pip install chromadb

In [55]:
vector_db = Chroma.from_documents(documents=docs, embedding=result)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [57]:
# Switching python ver to 3.9 as chroma is not supported on 3.11
# !sudo apt-get install python3.9

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9-minimal
Suggested packages:
  python3.9-venv binfmt-support
The following NEW packages will be installed:
  libpython3.9-minimal libpython3.9-stdlib python3.9 python3.9-minimal
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,846 kB of archives.
After this operation, 18.9 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-minimal amd64 3.9.22-1+jammy1 [837 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.9-minimal amd64 3.9.22-1+jammy1 [2,072 kB]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.9-stdlib amd64 3.9.22-1+jammy1 [1,844 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu ja

In [58]:
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

### Retreive the data

In [42]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [43]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

retriever = MultiQueryRetriever(
    vector_db.as_retriever(search_type="mmr"),
    llm=model,
    prompt=QUERY_PROMPT,

)


NameError: name 'vector_db' is not defined